In [ ]:
import os
import pandas as pd
from dotenv import load_dotenv

if os.getcwd().split("/")[-1] == "sandbox":
    os.chdir("../")
    
load_dotenv()

In [3]:
from food_co2_estimator.chains.recipe_extractor import get_recipe_extractor_chain
from food_co2_estimator.utils.url import get_url_text
recipe_extractor = get_recipe_extractor_chain(verbose=False)

In [4]:
from food_co2_estimator.utils.url import get_url_text

from food_co2_estimator.output_parsers.recipe_extractor import Recipe


url = "https://www.valdemarsro.dk/frikadeller/"
text = get_url_text(url)
recipe_extractor = get_recipe_extractor_chain(verbose=False)
recipe: Recipe = recipe_extractor.invoke(input=text)

In [5]:
from food_co2_estimator.chains.rag_so2_estimator import rag_co2_emission_chain
from food_co2_estimator.chains.translator import get_translation_chain

In [8]:
emission_chain = rag_co2_emission_chain()
translation_chain = get_translation_chain()

emission_chain_with_translation = translation_chain | emission_chain

emissions = emission_chain_with_translation.invoke(
    input={"inputs": recipe.ingredients, "language": "da"}
)

async test

In [12]:

recipe_extractor_chain = get_url_text | recipe_extractor
full_chain = recipe_extractor | emission_chain_with_translation


recipe = await recipe_extractor_chain.ainvoke(url)
emissions = await emission_chain_with_translation.ainvoke(
    input={"inputs": recipe.ingredients, "language": "da"}
)

In [ ]:
emissions